<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import pandas as pd
import numpy as np
import random
import sys
import os

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

In [2]:
# get the data
import requests

url = "https://www.gutenberg.org/files/100/100-0.txt"
data = requests.get(url)
text = data.text
print(len(text))

5777367


In [3]:
import re
regex = re.compile(r'[\n\r\t]')
text = regex.sub(' ', text)

In [4]:
# unique chars list
chars = list(set(text))

In [5]:
# peek at first 10
chars[:10]

['Z', 'H', '\x98', '¦', 'M', 'F', 'e', 'q', 'B', '\x9c']

In [6]:
# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [7]:
# Create the sequence data

maxlen = 80
# number of chars to go to the start of a new sequence
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 80 chars long
next_char = [] 

for i in range(0, len(encoded) - maxlen, step):
    """
    Loop 0 to the length of the text - 80. Increment by the step (5)
    """    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  1155458


In [8]:
# Create x & y

# creates a matrix of "falses"
# with a shape of len(sequences), max_len, len(chars)
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

# loop through entire text
for i, sequence in enumerate(sequences):
    # loop through one sequence 
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    # encode the target characters    
    y[i, next_char[i]] = 1

In [9]:
print(f'x shape: {x.shape}')
print(f'y shape: {y.shape}')

x shape: (1155458, 80, 111)
y shape: (1155458, 111)


In [10]:
# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
def sample(preds):
    # Lecture helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def on_epoch_end(epoch, _):
    # Lecture Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [14]:
# fit the model

model.fit(x, y,
          batch_size=512,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
2257/2257 [==============================] - ETA: 0s - loss: 2.3610 - accuracy: 0.3701
----- Generating text after Epoch: 0
----- Generating with seed: "hey did so; to the amazement of mine eyes,  That lookâd uponât.  Here comes "
hey did so; to the amazement of mine eyes,  That lookâd uponât.  Here comes roây ment thand wroje thes pont? w Wherse sh-mxent fatns andogse be d'd ond fokl.    HENCANTED. The lesis! in lbey   Ne.    Inde;  Ferl heprins.                           I that ind  Ifarud, ans noll?     ENDicG. Thet yof an tis in thom Olthes an, Forvame shean th mansâs led.                     Yeellay anull      Bukt foun be sor fuvr ceasmne ve kime the noup-nour you four ont; surs laus be s
2257/2257 [==============================] - 408s 181ms/step - loss: 2.3610 - accuracy: 0.3701
Epoch 2/10
2257/2257 [==============================] - ETA: 0s - loss: 1.9770 - accuracy: 0.4418
----- Generating text after Epoch: 1
----- Generating with seed: "give you?    M

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN